In [1]:
import sys
sys.path.append('../dgl_ptm')
import dgl_ptm
import os
os.environ["DGLBACKEND"] = "pytorch"

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
model = dgl_ptm.PovertyTrapModel(model_identifier='SA_test')

In [3]:
import torch
model.set_model_parameters(**{'number_agents': 20 , 
    'seed':42,
    'sigma_dist': {'type':'uniform','parameters':[0.05,1.94],'round':True,'decimals':1},
    'a_theta_dist': {'type':'uniform','parameters':[0.1,1],'round':False,'decimals':None},
    'sensitivity_dist':{'type':'uniform','parameters':[0.0,1],'round':False,'decimals':None},
    'capital_dist': {'type':'uniform','parameters':[0.1,10.],'round':False,'decimals':None}, 
    'alpha_dist': {'type':'normal','parameters':[1.08,0.074],'round':False,'decimals':None},
    'lambda_dist': {'type':'uniform','parameters':[0.05,0.94],'round':True,'decimals':1},
    'initial_graph_type': 'barabasi-albert',
    'initial_graph_args': {'seed': 1, 'new_node_edges':1},
    'device': 'cpu',
    'step_target':20,
    'steering_parameters':{'npath':'./agent_data.zarr',
                            'epath':'./edge_data', 
                            'ndata':[['degree','i_a','income','net_trade', 'tech_index','theta', 'wealth', 'wealth_consumption','weighted_degree'],['initial_only',['alpha','lambda','sigma','sensitivity']]],
                            'edata':None,
                            'mode':'w',
                            'capital_method':'past_shock',
                            'wealth_method':'weighted_transfer',
                            'income_method':'income_generation',
                            'tech_gamma': torch.tensor([0.3,0.35,0.45]),
                            'tech_cost': torch.tensor([0,0.15,0.65]),
                            'consume_method':'past_shock_bellman_consumption',
                            'nn_path': "/nn_data/both_PudgeSixLayer_1024/0723_110813/model_best.pth",
                            'adapt_m':torch.tensor([0,0.5,0.9]),
                            'adapt_cost':torch.tensor([0,0.25,0.45]),
                            'depreciation': 0.08,
                            'discount': 0.95,
                            'm_theta_dist': {'type':'beta','parameters':[4.13,0.07],'round':False,'decimals':None},
                            'del_method':'size',
                            'del_threshold':'balance',
                            'noise_ratio': 0.05,
                            'local_ratio': 0.25,
                            'homophily_parameter':1,
                            'characteristic_distance':3.33, 
                            'truncation_weight':1.0e-10,
                            'step_type':'ptm'}})

print(model.config.number_agents)  
print(model.config.steering_parameters)                        

20
edata=None epath='data_mode_B/edge_data' format='xarray' mode='w' ndata=['all_except', ['a_table', 'gamma', 'cost', 'zeros', 'ones']] npath='data_mode_B/agent_data.zarr' capital_method='past_shock' wealth_method='weighted_transfer' income_method='income_generation' consume_method='past_shock_bellman_consumption' nn_path='/nn_data/both_PudgeSixLayer_1024/0723_110813/model_best.pth' capital_update_method='default' characteristic_distance=3.33 homophily_parameter=1 adapt_m=tensor([0.0000, 0.5000, 0.9000]) adapt_cost=tensor([0.0000, 0.2500, 0.4500]) depreciation=0.08 discount=0.95 m_theta_dist=MThetaDist(type='beta', parameters=tensor([4.1300, 0.0700]), round=False, decimals=None) tech_gamma=tensor([0.3000, 0.3500, 0.4500]) tech_cost=tensor([0.0000, 0.1500, 0.6500]) del_method='size' del_threshold='balance' noise_ratio=0.05 local_ratio=0.25 truncation_weight=1e-10 step_type='ptm' data_collection_period=1 data_collection_list=None


In [4]:
model.initialize_model()
print (model.steering_parameters['modelTheta'] )
edgessrc,edgesdst=model.graph.edges(order='eid')
print(edgesdst[0:10])


Model torch seed set to 42
Using seed 1 for network creation with 1 edges requested.
20 agents initialized on cpu device
tensor([1.0000, 0.9869, 1.0000, 1.0000, 0.9998, 1.0000, 0.9909, 1.0000, 0.9967,
        0.9907, 1.0000, 1.0000, 1.0000, 0.9996, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 0.9674])
tensor([ 1,  2,  3,  4, 10, 13, 17,  5, 11,  6])


In [5]:
model.run()

#tensor([14,  6, 35, 22,  0, 21,  9, 62, 52,  2,  7, 12, 46,  8,  7, 12, 52, 67,
#        79, 52, 67, 21, 74, 67, 64])

performing step 0 of 20
Agent graph storage type: torch.int32
ATTENTION: No edge data collection requested for this simulation!
performing step 1 of 20
performing step 2 of 20
performing step 3 of 20
performing step 4 of 20
performing step 5 of 20
performing step 6 of 20
performing step 7 of 20
performing step 8 of 20
performing step 9 of 20
performing step 10 of 20
performing step 11 of 20
performing step 12 of 20
performing step 13 of 20
performing step 14 of 20
performing step 15 of 20
performing step 16 of 20
performing step 17 of 20
performing step 18 of 20
performing step 19 of 20


In [8]:
import zarr
import os
zarr_file_one = zarr.open('/Users/victoria/Documents/Scripts/Python/DGL-PTM/Sensitivity_Analysis/data_mode_A/agent_data.zarr', mode='r')
zarr_file_two = zarr.open('/Users/victoria/Documents/Scripts/Python/DGL-PTM/Sensitivity_Analysis/data_mode_B/agent_data.zarr', mode='r')
zarr_file_three = zarr.open('/Users/victoria/Documents/Scripts/Python/DGL-PTM/Sensitivity_Analysis/data_mode_B/agent_data_initial.zarr', mode='r')
print(zarr_file_one.tree())
print(zarr_file_two.tree())
print(zarr_file_three.tree())

#attributes=os.listdir('/Users/victoria/Documents/Scripts/Python/DGL-PTM/Sensitivity_Analysis/test_SA/agent_data.zarr')
#attributes = [item for item in attributes if item not in ['.zattrs','.zmetadata','.zgroup']]
#attributes = {key: None for key in attributes}
#print(attributes)
print(zarr_file_one['wealth'][0:2])
print(zarr_file_two['wealth'][0:2])

print(zarr_file_one['alpha'][0:2])
print(zarr_file_two['alpha'][0:2])
#Unnecessary:'a_table','m','tec','gamma','cost','zeros','ones', 'total_weight', "disposable_wealth"
#One-Time: 'alpha',"lambda","sigma","sensitivity"
#Every-Time: 'degree','i_a','income','net_trade', 'tech_index','theta', 'wealth', 'wealth_consumption','weighted_degree'


/
 ├── alpha (20, 20) float32
 ├── degree (20, 20) int64
 ├── disposable_wealth (20, 20) float32
 ├── i_a (20, 20) float32
 ├── income (20, 20) float32
 ├── lambda (20, 20) float32
 ├── m (20, 20) float32
 ├── net_trade (20, 20) float32
 ├── sensitivity (20, 20) float32
 ├── sigma (20, 20) float32
 ├── tec (20, 20) float32
 ├── tech_index (20, 20) int64
 ├── theta (20, 20) float32
 ├── total_weight (20, 20) float32
 ├── wealth (20, 20) float32
 ├── wealth_consumption (20, 20) float32
 └── weighted_degree (20, 20) float32
/
 ├── alpha (20, 20) float32
 ├── degree (20, 20) int64
 ├── disposable_wealth (20, 20) float32
 ├── i_a (20, 20) float32
 ├── income (20, 20) float32
 ├── lambda (20, 20) float32
 ├── m (20, 20) float32
 ├── net_trade (20, 20) float32
 ├── sensitivity (20, 20) float32
 ├── sigma (20, 20) float32
 ├── tec (20, 20) float32
 ├── tech_index (20, 20) int64
 ├── theta (20, 20) float32
 ├── total_weight (20, 20) float32
 ├── wealth (20, 20) float32
 ├── wealth_consumption (

In [7]:
ndata = ['all']
ndata = ['all_except',['a_table','m','tec','gamma','cost','zeros','ones', 'total_weight', "disposable_wealth"]]
ndata = [['all_except',['a_table','m','tec','gamma','cost','zeros','ones', 'total_weight', "disposable_wealth"]], ["initial_only",['alpha',"lambda","sigma","sensitivity"]]]
ndata = [['degree','i_a','income','net_trade', 'tech_index','theta', 'wealth', 'wealth_consumption','weighted_degree'], ["initial_only",['alpha',"lambda","sigma","sensitivity"]]]

timestep = 10

if ndata == ['all']:
    ndata = list(attributes.keys())
elif ndata[0] == 'all_except':
    ndata = list(attributes.keys() - ndata[1])
elif sum(1 for item in ndata if isinstance(item, list)) > 1:
    ndata_list = ndata
    initial_only=[]
    for specification in ndata_list:
        if specification == ['all']:
            raise ValueError('Use of "all" is not compatible with multiple data collection specification lists.')
        elif specification[0] == 'all_except':
            ndata = list(attributes.keys() - specification[1])
        elif specification[0] == 'initial_only':
            if timestep == 0:
                initial_only = specification[1]
        else:
            ndata = specification
    ndata=ndata+initial_only


print(ndata)


['degree', 'i_a', 'income', 'net_trade', 'tech_index', 'theta', 'wealth', 'wealth_consumption', 'weighted_degree']
